In [14]:
import json
def get_rate(json_fn, proc_name, region):
    with open(json_fn) as json_file:
        data = json.load(json_file)
        try:
            rate = data[region][proc_name]
        except:
            return 0
    return rate

In [21]:
year = '2018'

suff = 'test'
#bins = ['h_peak', 'h_sidebands', 'z_peak']
bins = ['ch1', 'ch2_h_sidebands', 'ch2_z_peak']
# suff = 'SR'
# bins = ['h_peak']

# suff = 'CR'
# bins = ['h_sidebands', 'z_peak']



ewk_name = {
    '2016':'ewk_lljj_mll105_160_ptj0',
    '2017':'ewk_lljj_mll105_160',
    '2018':'ewk_lljj_mll105_160'
}

sig_proc = ['ggh_amcPS', 'vbf_powhegPS',]

bkg_proc = {
    'ch1':[ewk_name[year], 'dy_m105_160_amc', 'dy_m105_160_vbf_amc',\
            'ttjets_dl', 
            'st_tw_top',\
            'st_tw_antitop',\
            'ttw', 'ttz',\
           'ww_2l2nu', 'wz_2l2q',\
            'wz_1l1nu2q',\
            'wz_3lnu', 'www','wwz','wzz','zzz'
           ],
    'ch2_h_sidebands':[ewk_name[year], 'dy_m105_160_amc', 'dy_m105_160_vbf_amc',\
            'ttjets_dl', 
            'st_tw_top',\
            'st_tw_antitop',\
            'ttw', 'ttz',\
           'ww_2l2nu', 'wz_2l2q',\
            'wz_1l1nu2q',\
            'wz_3lnu', 'www','wwz','wzz','zzz'
           ],
    'ch2_z_peak':[
            'ewk_lljj_mll50_mjj120', 'dy_0j', 'dy_1j', 'dy_2j',\
            'ttjets_dl', 
            'st_tw_top',\
            'st_tw_antitop',\
            'ttw', 'ttz',\
           'ww_2l2nu', 'wz_2l2q',\
            'wz_1l1nu2q',\
            'wz_3lnu', 'www','wwz','wzz','zzz'
           ],
}


proc = {b:sig_proc+bkg_proc[b] for b in bins}

rate_file = f'plots/postfit/norms_{year}.json'
input_file = {b:f'shapes_{b}_{year}_CRfit.root' for b in bins}

In [22]:
idx = []
rate = []
bin_names = []
proc_names = []

all_bins = []
all_observations = []

for bin_name in bins:
    all_bins.append(bin_name)
    all_observations.append("-1")
    for i, p in enumerate(proc[bin_name]):
        if p in sig_proc:
            idx.append(f"{-i:<25}")
        else:
            idx.append(f"{i:<25}")
        proc_names.append(f"{p:<25}")
        the_rate = get_rate(rate_file, p, bin_name)
        rate.append(f"{the_rate:<25}")
        bin_names.append(f"{bin_name:<25}")
                
    nproc = len(idx)
    nchannels = 1


all_bins_str = "  ".join(all_bins)
all_obs_str = "  ".join(all_observations)
bin_str = "".join(bin_names)
proc_str = "".join(proc_names)
idx_str = "".join(idx)
rate_str = "".join(rate)

datacard_name = f'plots/postfit/datacard_postfit_{suff}_{year}.txt'

In [23]:
datacard = open(datacard_name, 'w')
datacard.write(f"imax {len(bins)}\n")
datacard.write(f"jmax *\n")
datacard.write(f"kmax *\n")
datacard.write("---------------\n")
for bin_name in bins:
    datacard.write(f"shapes * {bin_name} {input_file[bin_name]} $PROCESS $PROCESS_$SYSTEMATIC\n")
datacard.write("---------------\n")
datacard.write(f"bin             {all_bins_str}\n")
datacard.write(f"observation     {all_obs_str}\n")
datacard.write("---------------\n")
datacard.write(f"{'bin':<40}       {bin_str}\n")
datacard.write(f"{'process':<40}       {proc_str}\n")
datacard.write(f"{'process':<40}       {idx_str}\n")
datacard.write(f"{'rate':<40}       {rate_str}\n")
datacard.write("---------------\n")

datacard.close()